### Load Data

Load raw data

In [1]:
import sys
from pathlib import Path
APP_PATH = str((Path.cwd().parent / 'app').resolve())

if APP_PATH not in sys.path:
    sys.path.append(APP_PATH)
    
%load_ext autoreload
%autoreload 2

In [2]:
from src.data import load_sessions

DATA_DIR = Path.cwd().parent / 'data'

sessions = load_sessions(DATA_DIR, cleanup=False)

Loading /home/jovyan/work/data/steinmetz_part0.npz...
Loading /home/jovyan/work/data/steinmetz_part1.npz...
Loading /home/jovyan/work/data/steinmetz_part2.npz...


In [4]:
import pickle

with open(DATA_DIR / "decoder_results.pickle", "rb") as f:
    dec_res = pickle.load(f)

In [5]:
import numpy as np

action_trials = dec_res[24]['action']['trial_numbers']
action_times = dec_res[24]['action']['decision_times']

vision_trials = dec_res[24]['vision']['trial_numbers']
vision_times = dec_res[24]['vision']['decision_times']

# common trials for action and vision
common_trials = np.intersect1d(action_trials, vision_trials)

In [6]:
# for common trials
vision_idx, action_idx = 500 + np.round([
    vision_times[np.in1d(vision_trials, common_trials)].mean(),
    action_times[np.in1d(action_trials, common_trials)].mean(),
])

### Do PCA for neural trajectories

Visual areas

In [3]:
import matplotlib.pyplot as plt
%matplotlib widget

plt.style.use('fivethirtyeight')

In [9]:
from src.pca import run_pca, smt_pca, map_pca
from src.selectors import get_selectors

# run pca on visual region

# Choose
sub = 24
sel = get_selectors(sessions[sub])
vis_dat = sessions[sub]['spks'][sel["NEURON_VISUAL"]]

print(vis_dat.shape)

W, V, variance_explained = run_pca(vis_dat[:,:,51:130])

# project data to pcs
pc_10ms = map_pca(W, V, vis_dat)

# get first two mean pcs and smooth them
n = 2500
pc_smt_ = np.zeros((pc_10ms.shape[0],n))

for i in range(pc_10ms.shape[0]):
    pc_smt = smt_pca(pc_10ms[i].mean(axis = 0),n)
    pc_smt_[i,:] = pc_smt

print(pc_10ms[0][1][51:].shape)

(94, 261, 250)
(199,)


In [10]:
# Map PCA for For vision
from src.trajectories import map_and_smooth, fit_tsne

sel['CHOICE_INCORRECT'] = ~sel['CHOICE_CORRECT']
outcomes = ['CHOICE_CORRECT' , 'CHOICE_INCORRECT']

contrast_levels = ["STIM_RIGHT_HIGH", "STIM_RIGHT_MEDIUM", "STIM_RIGHT_LOW", "STIM_RIGHT_NONE"]

# create two dictionaries
pc_vision_correct = {}
pc_vision_incorrect = {}

embed_correct = {}
embed_incorrect = {}

for outcome in outcomes:
    # get smoothed pc components for each
    print(f'pc_vision_{outcome[7:]}'.lower())
    for contrast in contrast_levels:
        # select data
        contrast_data = vis_dat[:, sel[contrast] & sel[outcome]]
        
        # save pc in dictionary
        if outcome == 'CHOICE_CORRECT':
            pc_vision_correct[contrast] = map_and_smooth(contrast_data, W, V)
            embed_correct[contrast] = fit_tsne(pc_vision_correct[contrast])

        elif outcome == "CHOICE_INCORRECT":
            pc_vision_incorrect[contrast] = map_and_smooth(contrast_data, W, V)
            embed_incorrect[contrast] = fit_tsne(pc_vision_incorrect[contrast])
        
        print(outcome, contrast, 'shape: ', contrast_data.shape)


pc_vision_correct
CHOICE_CORRECT STIM_RIGHT_HIGH shape:  (94, 43, 250)
CHOICE_CORRECT STIM_RIGHT_MEDIUM shape:  (94, 32, 250)
CHOICE_CORRECT STIM_RIGHT_LOW shape:  (94, 15, 250)
CHOICE_CORRECT STIM_RIGHT_NONE shape:  (94, 91, 250)
pc_vision_incorrect
CHOICE_INCORRECT STIM_RIGHT_HIGH shape:  (94, 22, 250)
CHOICE_INCORRECT STIM_RIGHT_MEDIUM shape:  (94, 13, 250)
CHOICE_INCORRECT STIM_RIGHT_LOW shape:  (94, 11, 250)
CHOICE_INCORRECT STIM_RIGHT_NONE shape:  (94, 34, 250)


In [11]:
# Visualization for vision, correct
from src.trajectories import traj_viz_continous, update_limits, add_colorbar

print('Vision Correct Trials')
fig, axs = plt.subplots(2,2, figsize=(10, 10), sharex=True, sharey=True)

titles = [ word+' Contrast' for word in ['High', 'Medium', 'Low', 'None']]

pc_indicse = [0, 1]

# random limits to start with
pc_xlim = [0,0.5]
pc_ylim = [0,0.5]

for i, contrast in enumerate(contrast_levels):
    traj_viz_continous(pc_vision_correct[contrast][pc_indicse[0]], pc_vision_correct[contrast][pc_indicse[1]], 
                       axs[i//2, i%2], 
                       name_x = 'PC 1', name_y = 'PC 2', 
                       title = titles[i])
    ys = plt.gca().get_ylim()
    xs = plt.gca().get_xlim()
    pc_xlim, pc_ylim = update_limits(pc_xlim, pc_ylim, xs, ys)

# Setting the values for all axes.
plt.setp(axs, xlim=pc_xlim, ylim=pc_ylim)
# plt.xticks(ticks=pc_xlim)
# plt.yticks(ticks=pc_ylim)

add_colorbar(axs[1,1], len(pc_vision_correct[contrast][0]))

Vision Correct Trials


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# Visualization for vision, incorrect
print('Vision Incorrect Trials')
fig, axs = plt.subplots(2,2, figsize=(10, 10), sharex=True, sharey=True)

titles = [ word+' Contrast' for word in ['High', 'Medium', 'Low', 'None']]

# random limits to start with
pc_xlim = [0,0.5]
pc_ylim = [0,0.5]

for i, contrast in enumerate(contrast_levels):
    traj_viz_continous(pc_vision_incorrect[contrast][pc_indicse[0]], pc_vision_incorrect[contrast][pc_indicse[1]], 
                       axs[i//2, i%2], 
                       name_x = 'PC 1', name_y = 'PC 2', 
                       title = titles[i])
    ys = plt.gca().get_ylim()
    xs = plt.gca().get_xlim()
    pc_xlim, pc_ylim = update_limits(pc_xlim, pc_ylim, xs, ys)

# Setting the values for all axes.
plt.setp(axs, xlim=pc_xlim, ylim=pc_ylim)
# plt.xticks(ticks=pc_xlim)
# plt.yticks(ticks=pc_ylim)

add_colorbar(axs[1,1], len(pc_vision_incorrect[contrast][0]))

Vision Incorrect Trials


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Using mean action decision time and mean vision decision time to color code trajectory.

In [13]:
from src.trajectories import traj_viz_discrete

pc01 = pc_vision_correct['STIM_RIGHT_HIGH'][0][500:]
pc02 = pc_vision_correct['STIM_RIGHT_HIGH'][1][500:]
traj_viz_discrete(pc01, pc02, vision_idx - 500, action_idx - 500, name_x = 'PC 1', name_y = 'PC 2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Visualize 2D neural trajectories with color coding based on timing. (Pre-stim, pre-action, post-action)

In [14]:
import math
from src.trajectories import traj_viz_anim

pc1 = pc_vision_correct['STIM_RIGHT_HIGH'][0][300: 1500: 3]
pc2 = pc_vision_correct['STIM_RIGHT_HIGH'][1][300: 1500: 3]
anim=traj_viz_anim(pc1, pc2, math.floor(len(pc1)* vision_idx / 2500), math.floor(len(pc2)* action_idx / 2500), 
                   name_x = 'PC 1', name_y = 'PC 2', color_type='discrete')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Motor Areas

In [15]:
# run pca on secondary motor region
mot_dat = sessions[sub]['spks'][sel["NEURON_MOTOR"]]

print(mot_dat.shape)

W_m, V_m, variance_explained_m = run_pca(mot_dat[:,:,0:200])

# project data to pcs
pc_mot = map_pca(W_m, V_m, mot_dat)

# get first two mean pcs and smooth them
n = 2500
pc_m_smt_ = np.zeros((pc_mot.shape[0],n))

for i in range(pc_mot.shape[0]):
    pc_m_smt = smt_pca(pc_mot[i].mean(axis = 0),n)
    pc_m_smt_[i,:] = pc_m_smt

print(pc_mot[0][1][51:].shape)

(80, 261, 250)
(199,)


In [16]:
# Map PCA For motor
actions = [
    ("GO_TRIALS", ~sel["STIM_NONE"] & (sessions[sub]['contrast_right'] != sessions[sub]['contrast_left'])),
    ("NOGO_TRIALS", sel["STIM_NONE"]),
]

pc_motor_correct = {}
pc_motor_incorrect = {}

for outcome in outcomes:
    # get smoothed pc components for each
    print(f'pc_motor_{outcome[7:]}'.lower())

    for action, selector in actions:
        # select data
        action_data = mot_dat[:, selector & sel[outcome]]
        print(outcome, action, 'shape: ', action_data.shape)
        
        # save pc in dictionary
        if outcome == 'CHOICE_CORRECT':
            pc_motor_correct[action] = map_and_smooth(action_data, W_m, V_m)
        elif outcome == "CHOICE_INCORRECT":
            pc_motor_incorrect[action] = map_and_smooth(action_data, W_m, V_m)

pc_motor_correct
CHOICE_CORRECT GO_TRIALS shape:  (80, 131, 250)
CHOICE_CORRECT NOGO_TRIALS shape:  (80, 50, 250)
pc_motor_incorrect
CHOICE_INCORRECT GO_TRIALS shape:  (80, 27, 250)
CHOICE_INCORRECT NOGO_TRIALS shape:  (80, 29, 250)


In [17]:
# for motor, correct
print('Motor Correct Trials')
fig, axs = plt.subplots(1,2, figsize=(10, 5), sharex=True, sharey=True)

titles = ['Go Trials', 'Nogo Trials']

# random limits to start with
pc_xlim = [0,0.3]
pc_ylim = [0,0.3]

for i, (action, _) in enumerate(actions):
    traj_viz_continous(pc_motor_correct[action][0], pc_motor_correct[action][1], 
                       axs[i%2], 
                       name_x = 'PC 1', name_y = 'PC 2', 
                       title = titles[i])
    ys = plt.gca().get_ylim()
    xs = plt.gca().get_xlim()
    pc_xlim, pc_ylim = update_limits(pc_xlim, pc_ylim, xs, ys)

# Setting the values for all axes.
plt.setp(axs, xlim=pc_xlim, ylim=pc_ylim)

add_colorbar(axs[1], len(pc_motor_correct[action][0]))

Motor Correct Trials


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
pc_m01 = pc_motor_correct['GO_TRIALS'][0][500:]
pc_m02 = pc_motor_correct['GO_TRIALS'][1][500:]
traj_viz_discrete(pc_m01, pc_m02, vision_idx - 500, action_idx - 500, name_x = 'PC 1', name_y = 'PC 2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
# for motor, incorrect
print('Motor Incorrect Trials')
fig, axs = plt.subplots(1,2, figsize=(10, 5), sharex=True, sharey=True)

titles = ['Go Trials', 'Nogo Trials']

# random limits to start with
pc_xlim = [0,0.3]
pc_ylim = [0,0.3]

for i, (action, _) in enumerate(actions):
    traj_viz_continous(pc_motor_incorrect[action][0], pc_motor_incorrect[action][1], 
                       axs[i%2], 
                       name_x = 'PC 1', name_y = 'PC 2', 
                       title = titles[i])
    ys = plt.gca().get_ylim()
    xs = plt.gca().get_xlim()
    pc_xlim, pc_ylim = update_limits(pc_xlim, pc_ylim, xs, ys)

# Setting the values for all axes.
plt.setp(axs, xlim=pc_xlim, ylim=pc_ylim)

add_colorbar(axs[1], len(pc_motor_incorrect[action][0]))

Motor Incorrect Trials


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Visualize TSNE results. Not important. 

In [20]:
# Visualization for vision, correct
print('Vision Correct Trials')
fig, axs = plt.subplots(2,2, figsize=(10, 10), sharex=True, sharey=True)

titles = [ word+' Contrast' for word in ['High', 'Medium', 'Low', 'None']]

pc_indicse = [ 5, 6]

# random limits to start with
pc_xlim = [0,0.5]
pc_ylim = [0,0.5]

for i, contrast in enumerate(contrast_levels):
    traj_viz_continous(embed_correct[contrast][:, 0], embed_correct[contrast][:, 1], 
                       axs[i//2, i%2], 
                       name_x = 'TSNE 1', name_y = 'TSNE 2', 
                       title = titles[i])
    ys = plt.gca().get_ylim()
    xs = plt.gca().get_xlim()
    pc_xlim, pc_ylim = update_limits(pc_xlim, pc_ylim, xs, ys)

# Setting the values for all axes.
plt.setp(axs, xlim=pc_xlim, ylim=pc_ylim)
# plt.xticks(ticks=pc_xlim)
# plt.yticks(ticks=pc_ylim)

add_colorbar(axs[1,1], len(pc_vision_correct[contrast][0]))


fig, axs = plt.subplots(2,2, figsize=(10, 10), sharex=True, sharey=True)

titles = [ word+' Contrast' for word in ['High', 'Medium', 'Low', 'None']]

pc_indicse = [ 5, 6]

# random limits to start with
pc_xlim = [0,0.5]
pc_ylim = [0,0.5]

for i, contrast in enumerate(contrast_levels):
    traj_viz_continous(embed_incorrect[contrast][:, 0], embed_incorrect[contrast][:, 1], 
                       axs[i//2, i%2], 
                       name_x = 'TSNE 1', name_y = 'TSNE 2', 
                       title = titles[i])
    ys = plt.gca().get_ylim()
    xs = plt.gca().get_xlim()
    pc_xlim, pc_ylim = update_limits(pc_xlim, pc_ylim, xs, ys)

# Setting the values for all axes.
plt.setp(axs, xlim=pc_xlim, ylim=pc_ylim)
# plt.xticks(ticks=pc_xlim)
# plt.yticks(ticks=pc_ylim)

add_colorbar(axs[1,1], len(pc_vision_correct[contrast][0]))
# Visualization for vision, correct
print('Vision Incorrect Trials')

Vision Correct Trials


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Vision Incorrect Trials
